## Toxic comment classification
### Import library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc, f1_score

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from scipy import sparse

## Read data set

In [2]:
toxic = pd.read_csv('toxicity_data/train.csv') #there's also a test dataset but it doesn't have labels b/c kaggle.
#test.fillna(' ',inplace=True) - this line can clean up some issues in the test data if you do use it
print('Number of rows and columns in the train data set:',toxic.shape)

#unlabeled data
incel_df = pd.read_csv('new_IncelTears_posts.csv')
slate_df = pd.read_csv('new_slatestarcodex_posts.csv')

raw_toxic = toxic
small_toxic = toxic # this decreases time from 50 seconds to less
#small_toxic = toxic # this is just so I don't have to rename rest of cells

#turn multi-class into single class classifier
target_col = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
y = small_toxic[target_col]
y['sum'] = y.sum(axis=1).astype(bool).astype(int)

X_train, X_holdout, y_train, y_holdout = train_test_split(small_toxic, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


Number of rows and columns in the train data set: (159571, 8)


## Text preprocessing

[source: ](https://www.kaggle.com/him4318/easy-and-fast-lb-044) 

Term Frequency Inverse Document  Frequency Vectorizer 

In [3]:
vect_word = TfidfVectorizer(max_features=20000, lowercase=True, analyzer='word',
                        stop_words= 'english',ngram_range=(1,3),dtype=np.float32)
tr_vect = vect_word.fit_transform(X_train['comment_text'])
ts_vect = vect_word.transform(X_test['comment_text'])

incel_vect = vect_word.transform(incel_df['title'])
slate_vect = vect_word.transform(slate_df['title'])

#took 50 seconds on 150k samples

## LR Model

In [4]:
lr = LogisticRegression(C=2,random_state = 42,class_weight = 'balanced')
lr.fit(tr_vect,y_train['sum'])

pred =  lr.predict(ts_vect)
print('\nConfusion matrix\n',confusion_matrix(y_test['sum'],pred))
print(classification_report(y_test['sum'],pred))


Confusion matrix
 [[21668  1211]
 [  428  2225]]
              precision    recall  f1-score   support

           0       0.98      0.95      0.96     22879
           1       0.65      0.84      0.73      2653

   micro avg       0.94      0.94      0.94     25532
   macro avg       0.81      0.89      0.85     25532
weighted avg       0.95      0.94      0.94     25532



In [5]:
f1_score(y_test['sum'],pred)  

0.7308260798160617

In [11]:
#RUN ONCE TO SAVE MODEL
# import pickle
# with open('lr_model.pickle', 'wb') as handle:
#     pickle.dump(lr, handle, protocol=pickle.HIGHEST_PROTOCOL)

### IMPORTS PICKLED LR MODEL
# with open('lr_model.pickle', 'rb') as handle:
#     lr = pickle.load(handle)

#RUN ONCE TO SAVE FIT VECTORIZER
# import pickle
# with open('fit_vect.pickle', 'wb') as handle:
#      pickle.dump(vect_word, handle, protocol=pickle.HIGHEST_PROTOCOL)



## Take a look at one of our negative & positive subreddits

In [6]:
incel_preds = lr.predict(incel_vect)
print(f'Percentage of Incel titles predicted as toxic {incel_preds.sum()/incel_preds.shape[0]}')

slate_preds = lr.predict(slate_vect)
print(f'Percentage of Slate titles predicted as toxic {slate_preds.sum()/slate_preds.shape[0]}')

score_ratio = (incel_preds.sum()/incel_preds.shape[0])/(slate_preds.sum()/slate_preds.shape[0]) #good subreddit 13x better.
print (f'Slate is {round(score_ratio,2)}x better)')

Percentage of Incel titles predicted as toxic 0.30792377131394183
Percentage of Slate titles predicted as toxic 0.06820461384152457
Slate is 4.51x better)


## Let's eyeball the results too

In [7]:
incel_df[np.isin(incel_preds, 0)]['title'].values[:10] #these are the ones it said were ok.

array(['Another one thinking he is a genious', '"But were nonviolent"',
       'Probably a LARP as I imagine the only thing he actually lifts is Cheeto packets. However if true, I hope the next woman be tries it with gives him what he deserves...',
       'Because of course he’s entitled to a woman’s body if someone else has had it.',
       'They bring so much of their unhappiness upon themselves',
       'incel worried about an epidemic of "open mouthed skinny framed" guys',
       '"clothes. But her bone structure is terrible, if she was born male, she would be extremely repulsive and for sure involuntary adult virgin"',
       '“Females are the problem, not males.”',
       'Incel goes outside and overhears a conversation that had nothing to do with him and takes personal offense.',
       'Incel blaming genetics while at the same time refusing to work on himself, you ain’t gonna get anywhere by doing that.'],
      dtype=object)

Still some pretty bad stuff getting missed.

In [8]:
slate_df[np.isin(slate_preds, 1)]['title'].values[:10] #these are the ones it said were ok.

array(['In Defense of Inclusionism',
       'Following up on "College Has Been Oversold"',
       '*The Elephant in the Brain* Discussion Questions',
       'How an Aspiring ‘It’ Girl Tricked New York’s Party People — and Its Banks',
       'Public Education’s Dirty Secret',
       "Plummeting insect numbers 'threaten collapse of nature' | Environment | The Guardian",
       'Nature: Human Mind Control of Rat Cyborg’s Continuous Locomotion with Wireless Brain-to-Brain Interface',
       'The no-nonsense guide for people who think they might have an eating disorder',
       'Why do you think humanity should exist?', 'Hungry Trolls'],
      dtype=object)

Still some fine stuff getting misclassified